In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import random
import pickle
import re

import spacy
import json
from spacy import displacy
import collections
from collections import Counter
from collections import OrderedDict
from tqdm.notebook import tqdm as tqdm_notebook
from itertools import islice

nlp = spacy.load('en_core_web_trf')

In [2]:
glossary = pickle.load(open('../../data/glossaries/FNA_glossary.pkl', 'rb'))
glossary['leaf'] += ['leave']

In [3]:
#glossary_FNA['Flower']

In [4]:
compound_list = [
    'fertile', 'sterile',
    'male', 'female', 'bisexual', 'hermaphroditic', 
    'basal', 'developed', 
    'primary', 'secondary', 'main',
    'upper', 'lower', 'greater', 'dorsal', 'alternate', 'lesser', 'apex', 'outer',
    'central', 'outermost', 'outer', 'inner', 'uppermost', 'median', 'dorsal', 'central', 'lateral',
    'young', 'mature', 'individual', 
    'opposite', 'single', 'paired',
]

rubbish_list = [
    '.', ',', '-', '..', '...', '', 
]


In [5]:
#data = pickle.load(open('../../data/PlantNet/descriptions_raw.pkl', 'rb'))
data = pickle.load(open('../../data/description/04_TRAIN_0000000-0014557_PLANTS.pkl', 'rb'))

In [6]:
measurements_list = [
    'mm', 'cm', 'm', 'km',
    'milimeter', 'centimeter', 'meter', 'kilometer',
    'milimetre', 'centimetre', 'metre', 'kilometre',
    'inch', 'foot', 'yard', 'mile',
    'wide', 'long', 'broad', 'tall',
    'length', 'form',
]

In [37]:
def text_preparation(species, text):
    cleaners = [(r'(?<!\d)\.(?!\d)', ' '),
                (r'\s×\s', ' times '),
                (r'\s+c\s+', ' '),
                (r'â\x80\x93', ' to '),
                (r'\xa0', ' '),
                (r'\x97', ''),
                (r'\s{2,}', ' '),
                (r'(\D)(\.)', r'\1 '),
                (r'(\d)(\.)(\D)', r'\1 \3'),
                (r'(long,)', r'long and'),
                (r'(wide,)', r'wide and'),
               ]
    
    species_parts = species.split()
    candidates = [' '.join(species_parts[:idx+1]) for idx, _ in enumerate(species_parts)]
    #candidates += [
    #        f'{species_parts[0][0]}. {species_parts[1]}'
    #    ]
    candidates.reverse()
    for candidate in candidates:
        try:
            text = re.sub(candidate, 'the species', text)
        except:
            continue # Skip species with brackets for now
    for (cleaner, replacement) in cleaners:
        text = re.sub(cleaner, replacement, text)    
    text = f'{text.strip()}.'
    return text.capitalize()


def dict_sentence(t):
    sentence_dict = {}
    for child in t.children:
        # Exceptions
        if (
            child.dep_ not in [
                'det', 'cc', 'punct', 
                'poss', 
                'nmod', 'ccomp', # Gives to many errors
                'conj',
                'prt', # Verb things
            ]
            and child.pos_ not in [
                'DET', 'PUNCT',
                'PART',
                'ADV',
                'SCONJ',
                'PRON',
            ]
            and child.lemma_ not in [
                'on', 'of', 'as', 'from', 'by', 'onto',
            ]
        ):
            items = dict_sentence(child)
            sentence_dict[child] = items 
    return sentence_dict

def undict_to_tuples(d, acc = []):
    if d == {}:
        yield acc
    else:
        for k, v in d.items():
            yield from undict_to_tuples(v, acc + [k,])
            
def undict_to_pairs(d):
    for k,v in d.items():
        for subk in v:
            yield (k, subk)
        yield from undict_to_pairs(v)

def dict_sentence_parent(t):
    if t.dep_ in [
        'nsubj', 'nsubjpass', 'relcl',
    ]:
        parent = next(tok for tok in t.ancestors)
        parent_dict = dict_sentence(parent)
        del parent_dict[t]
        return parent_dict

def update_nested_dict(main_dict, new_dict):
    for name, rc_dict in new_dict.items():
        main_dict.setdefault(name, {}).update(rc_dict)
    return main_dict       

def extract_compounds(t, doc):
    head = None
    if t.dep_ == 'compound':
        t = next(t.ancestors)
    indices = [child.i for child in t.children
               if child.dep_ == 'compound'
               or child.lemma_ in compound_list
               and child.i < t.i]
    indices.append(t.i)
    indices.sort(reverse=True)
    compounds  = []
    for idx in indices:
        compounds.append(doc[idx : t.i + 1])
    return compounds    
            
            
def check_existance(t):
    item = None
    for mainpart in glossary.keys():
        if t.lemma_ in compound_list:
            item = None
        #elif t.pos_ != 'NOUN':
        elif t.pos_ not in ['NOUN', 'PROPN']:
            item = None
        elif t.lemma_.lower().strip() in glossary[mainpart]:
            item = mainpart            
    return item


def clean_compounds(item_list, doc):
    new_item_list = []
    new_item_list.append(item_list[0])
    for item in item_list[1:]:
        item_orig = item
        if type(item) == spacy.tokens.token.Token:
            item = item
        else:
            item = item.root
        #if item.dep_ == 'prep':
        #    new_item_list += item_list[1:]
        #    break
        if (
            item.lemma_ in compound_list 
            or item.dep_ == 'compound'
        ):
            continue
        elif item.pos_ == 'NOUN':
            compound = extract_compounds(item, doc)[-1]
            if len(compound) == 1:
                #print(compound)
                compound = compound.root
            new_item_list.append(item_orig)
        else:
            new_item_list.append(item_orig)
    return new_item_list

def clean_measurements(info_list):
        
    new_item_list = []
    # Retok
    for item_list in info_list:
        nums = [t.dep_ for t in item_list if type(t) == spacy.tokens.token.Token if t.pos_ == 'NUM']
        if len(nums) > 1:
            temp = []
            for item, future in zip(item_list, item_list[1:]):        
                if type(item) == spacy.tokens.span.Span:
                    temp.append(item)
                elif item.pos_ == 'NUM' and future.pos_ == 'NUM':
                    new_item_list.append(temp + [item])
                    new_item_list.append(temp + [future])
                else:
                    temp.append(item)        
        else:
            new_item_list.append(item_list)   
    return new_item_list


def clean_prepositions_of(info_list):
    new_info_lists = []
    for info in info_list:
        new_item_list = []
        used = []
        for item in info:
            item_orig = item
            if type(item) == spacy.tokens.token.Token:
                item = item
            else:
                item = item.root
            prep_of = next((t for t in item.children if t.lemma_ == 'of'), None)
            if prep_of:
                child = next((t for t in prep_of.children if t.dep_ == 'pobj'), None)
            if prep_of and child:
                #print(item, prep_of, child)
                new_item_list.append(doc[item.i : child.i + 1])
            else:
                new_item_list.append(item_orig)
        new_info_lists.append(new_item_list)
    return new_info_lists


def clean_prepositions_on(info_list):
    new_info_lists = []
    #print(info_list)
    for info in info_list:
        new_item_list = []
        used = []
        for item in info:
            item_orig = item
            #print(item)
            if type(item) == spacy.tokens.token.Token:
                item = item
            else:
                item = item.root
            prep = next((t for t in item.children if t.lemma_ == 'on'), None)
            #print(prep_on)
            if prep:
                child = next((t for t in prep.children if t.dep_ == 'pobj'), None)
            if prep and child:
                #print(item, prep, child)
                new_item_list.append(doc[item.i : child.i + 1])
            else:
                new_item_list.append(item_orig)
        new_info_lists.append(new_item_list)
    return new_info_lists



    


def clean_prepositions_in(info_list):
    new_info_lists = []
    for info in info_list:
        new_item_list = []
        used = []
        for item in info:
            if item.lemma_ == 'in':
                break
            new_item_list.append(item)
        new_info_lists.append(new_item_list)
    return new_info_lists


def clean_prepositions_with(info_list):
    new_info_lists = []
    for info in info_list:
        new_item_list = []
        used = []
        for item in info:
            if item.lemma_ == 'with':
                continue
            new_item_list.append(item)
        new_info_lists.append(new_item_list)
    return new_info_lists

        
def create_relation(compounds, item_list):    
    subjects = item_list[:-1]
    objects  = item_list[1:]
    relation = None
    triples = []
    
    #print(compounds[-1])
    
    # Compound
    compound = compounds[-1][-1]
    # Flatten
    compounds = list(sum(compounds, ()))
    
    for sub, obj in zip(subjects, objects):
        relation = f'{compound} temp'
        if sub.lemma_ in compounds:
            sub = compound
        try:
            obj_text = obj.lemma_
        except:
            obj_text = obj
        try:
            sub_text = sub.lemma_
        except:
            sub_text = sub
        if obj_text in sub_text:
            continue
        triples.append((sub, relation, obj))
    return triples
    
def extract_triples(doc):
    # Speed up the extraction
    previous = []
    AIKEs_list = []
    triples = []
    for t in doc:
        part = check_existance(t)
        #print(part, t)
        if part:
            # Create temp list for storing compounds
            compounds_temp = []
            compounds_temp.append(('species', 'has_main_part', part.capitalize()))           
            compound = part.capitalize()
            #print(compound)
            for new_compound in extract_compounds(t, doc):
                if type(compound) == str:
                    compounds_temp.append((compound, 'has_sub_part', new_compound.lemma_))
                else:
                    compounds_temp.append((compound.lemma_, 'has_sub_part', new_compound.lemma_))
                compound = new_compound
                #print(compound)
            # Reset T
            t = compound.root
            # Get child dict
            child_dict = {compound: dict_sentence(t)}
            # Get parent dict
            parent_dict = {compound: dict_sentence_parent(t)}
            # Update if exists
            if parent_dict[compound]:
                # Add dicts together
                sentence_dict = update_nested_dict(child_dict, parent_dict)
                #print(sentence_dict)
            else:
                sentence_dict = child_dict
            # List dict into tuples
            info_lists = list(undict_to_tuples(sentence_dict))
            info_lists = clean_measurements(info_lists)
            info_lists = clean_prepositions_of(info_lists)
            info_lists = clean_prepositions_in(info_lists)
            info_lists = clean_prepositions_on(info_lists)
            info_lists = clean_prepositions_with(info_lists)
            #clean_measurements(info_lists)

            #print('end', t, info_lists)
            for info in info_lists:
                # Skip no info
                if len(info) <= 1:
                    continue
                info = clean_compounds(info, doc)
                triples.extend(compounds_temp)
                triples.extend(create_relation(compounds_temp, info))
            
    return list(dict.fromkeys(triples))

In [38]:
#string = 'Fertile lemma lanceolate in profile, or oblong in profile, 2.5-3.8 milimeter long, membranous, mid-green and grey, 3 -veined.'
string = 'The leaves are 2 to 4 meters long.'
doc = nlp(string)
extract_triples(doc)

[('species', 'has_main_part', 'Leaf'),
 ('Leaf', 'has_sub_part', 'leave'),
 ('leave', 'leave temp', long),
 (long, 'leave temp', meters),
 (meters, 'leave temp', 4),
 (meters, 'leave temp', 2),
 (4, 'leave temp', to)]

In [9]:
displacy.render(doc)

In [ ]:
species_list = list(data.keys())
species = species_list[535]

In [ ]:
kn_data = []
for text in tqdm_notebook(data[species][0:16]):
    text = text_preparation(species, text)
    doc = nlp(text)
    #if len(doc) > 20:
    #    continue
    kn_data.extend(extract_triples(doc))

In [ ]:
doc

In [ ]:
kn_data

In [ ]:
displacy.render(doc)

In [ ]:
root = next(t for t in doc if t.dep_ == 'ROOT')
print(root, root.dep_)

In [ ]:
doc.text

In [ ]:
re.sub(r'(long,)', r'long and', doc.text)

# VIZ

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from netgraph import Graph
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
#descriptions

In [ ]:
source   = []
relation = []
target   = []


for (sub, rel, obj) in kn_data:
    if type(sub) != str:
        sub = sub.lemma_
    if type(obj) != str:
        obj = obj.lemma_
    if sub ==  obj:
        continue
    source.append(sub)
    relation.append(rel)
    target.append(obj)

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relation})

In [ ]:
#kn_data

In [ ]:
nodes = [(source, target) for source, target in zip(kg_df['source'].values, kg_df['target'].values)]
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.Graph())


node_labels = {node : node for idx, node in enumerate(G)}
edge_labels = dict(zip(list(zip(kg_df.source, kg_df.target)),
                  kg_df['edge'].tolist()))

node_size = {}
node_color = {}

size = 1.5

for node in node_labels:
    if node == 'species':
        node_size[node] = 3.5/size
        node_color[node] = 'darkgreen'
    elif node[0].isupper():
        node_size[node] = 2/size
        node_color[node] = 'white'
    else:
        node_size[node] = 1./size
        node_color[node] = 'white'
        
pos = nx.spring_layout(G, k = 0.08, iterations=5000, seed=3, scale=0.3, center=(0,0), dim=2)

In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))
Graph(nodes, 
      #node_layout='spring', edge_layout='curved', 
      #node_layout=pos, edge_layout='straight', 
      node_layout='spring', edge_layout='straight',
      arrows=True, node_zorder=3, #edge_zorder=1,
      node_labels=node_labels, 
      node_label_offset=0.02, 
      #edge_labels=edge_labels,
      node_label_fontdict=dict(size=18, rotation=0, ha='center', clip_on=False), node_edge_width=0.2,
      node_size=node_size,  node_color=node_color, #edge_labels=edge_labels,
      edge_width=0.2, edge_label_fontdict=dict(size=10,),
      #node_layout_kwargs=dict(node_size=1, total_iterations=20),
      ax=ax)

In [ ]:
doc = nlp('legs')

In [ ]:
displacy.render(doc)

In [ ]:
len(doc)

In [ ]:
doc[0].pos_